<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/ways_of_convert_rubert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

Этот ноутбук для бекендеров, которым дали модельку и сказали деплоить так, чтобы она быстро работала. И больше ничего не дали, кроме модельки
	
  (・_・ヾ

Устанавливаем нужные зависимости

In [40]:
!pip install onnx transformers onnxruntime folium==0.2.1 optimum[onnxruntime]

     |████████████████████████████████| 4.0 MB 12.3 MB/s 
     |████████████████████████████████| 5.2 MB 36.3 MB/s 
     |████████████████████████████████| 69 kB 8.0 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 40.1 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 325 kB 45.7 MB/s 
     |████████████████████████████████| 212 kB 53.3 MB/s 
     |████████████████████████████████| 136 kB 54.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.4 MB/s 
     |████████████████████████████████| 127 kB 40.8 MB/s 
     |████████████████████████████████| 271 kB 58.5 MB/s 
     |██████

Фиксируем версии библиотек

In [41]:
!pip freeze > req.txt

Импорты

In [42]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from torch.nn.utils import prune
from optimum.onnxruntime import ORTQuantizer
import tensorrt as trt
from tensorrt import ICudaEngine, IExecutionContext
from tensorrt.tensorrt import (
    Builder,
    IBuilderConfig,
    IElementWiseLayer,
    ILayer,
    INetworkDefinition,
    IOptimizationProfile,
    IReduceLayer,
    Logger,
    OnnxParser,
    Runtime,
)

Качаем датасет, на котором будем проверять качество модели

In [ ]:
!wget https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv

# Об нейросети

В качестве подопытного будем использовать [rubert-base-cased-sentiment](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для классификации русских предложений. Данная нейросеть предсказывает 3 метки класса, в зависимости от тона предложения - позитивное, негативное или нейтральное

Запускаем нейросеть как есть

In [51]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [44]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

Проверим время инференса модели

In [45]:
%%time
predict(text)

CPU times: user 131 ms, sys: 171 ms, total: 302 ms
Wall time: 403 ms


2

Проверим качество модели. Для проверки качества будем использовать один из датасетов, на котором обучалась модель, а именно [этот](https://github.com/sismetanin/rureviews)

In [ ]:
df = pd.read_csv('/content/women-clothing-accessories.3-class.balanced.csv', delimiter='\t')
df.head()

Для удобства немного изменим датасет - заменим метки класса на цифровые значения и выберем 1000 рандомных строк

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:1000]
mapping = {'negative': 2, 'positive': 1, 'neautral':0}
df = df.replace({'sentiment': mapping})
df.head()

Считаем качество

In [ ]:
texts = list(df['review'])
labels = list(df['sentiment'])

In [ ]:
predictions = [predict(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


Сохраним оригинальную модель и посмотрим на ее вес

In [ ]:
torch.save(model, 'output/original.pt')

In [ ]:
!du -shc output/original.pt

922M	output/original.pt
922M	total


# ONNX

Формат Open Neural Network Exchange (ONNX) обеспечит общий способ представления данных, используемых в нейронных сетях. Большинство платформ имеют сегодня собственный специфический формат моделей, которые способны работать с моделями других платформ только при использовании специальных инструментов преобразования форматов.

ONNX позволит осуществлять свободный обмен информацией, которой обладают модели, без процедуры преобразования. Модель, обученную на одной платформе, можно будет использовать и на другой платформе. Также можно будет модель, обученную на одном фреймворке, перенести на другой фреймворк.

Перевести модель в ONNX можно несколькими способами:

1. Есть библиотека transforms, [где все почти из коробки](https://huggingface.co/docs/transformers/serialization)

In [46]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [47]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [48]:
!mkdir -p output/onnx_transforms

In [52]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert-base-cased-sentiment.onnx"),
        opset=11)

Пробуем запустить в ONNX и посмотреть время инференса

In [ ]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment.onnx", sess_options, providers)

In [ ]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [ ]:
%%time
predict_onnx(text)

CPU times: user 89.7 ms, sys: 744 µs, total: 90.5 ms
Wall time: 89.5 ms


2

Считаем качество

In [ ]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


Посмотрим на вес модели

In [ ]:
!du -shc output/onnx_transforms/*

679M	output/onnx_transforms/rubert-base-cased-sentiment.onnx
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала на порядок выше, метрики качества не изменились.Вес модели уменьшился на 300 мб

# TorchScript

TorchScript — инструмент, который позволяет с помощью пары строк кода и нескольких щелчков мыши сделать из пайплайна на питоне отчуждаемое решение, которое можно встроить в систему на C++. А еще она будет на python работать быстрее из-за jit компиляции. В библиотеке transformers так же [есть почти из коробки](https://huggingface.co/docs/transformers/serialization#torchscript)

Первым шагом нужно перезагрузить модель, добавив флаг torchscript=True

In [ ]:
model_torchscript = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, torchscript=True)

In [ ]:
inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
traced_model = torch.jit.trace(model_torchscript, [torch.tensor(x) for x in inputs.values()])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
!mkdir -p output/torchscript

In [ ]:
torch.jit.save(traced_model, "output/torchscript/rubert-base-cased-sentiment_traced.pt")

Пробуем предиктить

In [ ]:
@torch.no_grad()
def predict_torchscript(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = traced_model(**inputs)[0]
    predicted = torch.nn.functional.softmax(outputs, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [ ]:
%%time
predict_torchscript(text)

CPU times: user 110 ms, sys: 1.92 ms, total: 111 ms
Wall time: 116 ms


2

Считаем качество модели

In [ ]:
predictions = [predict_torchscript(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7758420684835778, recall: 0.7625989008405051, f1score: 0.7586448839591


 Посмотрим на вес модели

In [ ]:
!du -shc output/torchscript/*

679M	output/torchscript/rubert-base-cased-sentiment_traced.pt
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала немного выше, метрики качества не изменились.Вес модели изменился на 300мб примерно. Лучше, чем ничего

# Прунинг модели

Model Pruning — обрезание избыточных частей сети для ускорения инференса без потери точности. Наглядно — откуда, сколько и как можно вырезать.

Есть два варианта, как прунить модель.

1 вариант - библиотека [nn_pruning](https://github.com/huggingface/nn_pruning) от HuggingFace

2 вариант - делать через torch.nn.utils.prune. В качестве примера есть данный [ноутбук](https://github.com/Huffon/nlp-various-tutorials/blob/master/pruning-bert.ipynb)

[Ссылка](https://aclanthology.org/2020.repl4nlp-1.18.pdf) на почитать про прунинг модели BERT

Запруним encoder слои

In [ ]:
pruned_model = model

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (pruned_model.bert.encoder.layer[i].attention.self.key, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.query, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

Выведем, что получилось

In [ ]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0)

    denominator += pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 18.59%
Sparsity in Layer 1-th query weightt: 18.67%
Sparsity in Layer 1-th value weight: 26.79%

Sparsity in Layer 2-th key weight: 18.77%
Sparsity in Layer 2-th query weightt: 18.33%
Sparsity in Layer 2-th value weight: 25.69%

Sparsity in Layer 3-th key weight: 20.08%
Sparsity in Layer 3-th query weightt: 19.58%
Sparsity in Layer 3-th value weight: 23.53%

Sparsity in Layer 4-th key weight: 18.77%
Sparsity in Layer 4-th query weightt: 18.49%
Sparsity in Layer 4-th value weight: 24.32%

Sparsity in Layer 5-th key weight: 18.40%
Sparsity in Layer 5-th query weightt: 18.36%
Sparsity in Layer 5-th value weight: 23.00%

Sparsity in Layer 6-th key weight: 18.32%
Sparsity in Layer 6-th query weightt: 17.84%
Sparsity in Layer 6-th value weight: 21.55%

Sparsity in Layer 7-th key weight: 18.54%
Sparsity in Layer 7-th query weightt: 18.06%
Sparsity in Layer 7-th value weight: 22.07%

Sparsity in Layer 8-th key weight: 18.60%
Sparsity in Layer 8-th query weigh

Предиктим на запруненной модели

In [ ]:
@torch.no_grad()
def predict_pruned(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = pruned_model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [ ]:
%%time
predict_pruned(text)

CPU times: user 148 ms, sys: 8 ms, total: 156 ms
Wall time: 155 ms


2

In [ ]:
!mkdir -p output/pruning

In [ ]:
torch.save(pruned_model, 'output/pruning/rubert-base-cased-sentiment_pruned.pt')

Посчитаем качество модели

In [ ]:
predictions = [predict_pruned(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7673895104392958, recall: 0.759069038827347, f1score: 0.7562227505683002


 Посмотрим на вес модели

In [ ]:
!du -shc output/pruning/*

922M	output/pruning/rubert-base-cased-sentiment_pruned.pt
922M	total


Вывод - вес модели не поменялся, качество упало чуть-чуть по сравнению с оригинальной моделью, но не критично. Модель стала работать быстрее, но ценой небольшой потери качества

# Квантизация

Квантизация означает уменьшение численной точности весов модели. Один из популярных методов — k-means квантизация. Имея веса модели в матрице W с десятичными числами, веса кластеризуются с помощью k-means в N кластеров. Затем матрица W трансформируется в матрицу целых чисел от 1 до N, каждое из которых является указателем к центру кластера. Так можно сжать каждый элемент изначальной матрицы из 32-битного десятичного числа в log(N)-битные целые числа.

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)


Делаем тоже по лучшим гайдам https://github.com/huggingface/optimum



Динамическая квантизация не требует ничего, поэтому она самая простая

In [ ]:
!mkdir -p output/quantization

In [ ]:
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained("blanchefort/rubert-base-cased-sentiment", feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert-base-cased-sentiment.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx')

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [ ]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx", sess_options, providers)

In [ ]:
%%time
predict_onnx(text)

CPU times: user 55.9 ms, sys: 856 µs, total: 56.7 ms
Wall time: 57.8 ms


2

Считаем качество

In [ ]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7634891164434495, recall: 0.7515760349283417, f1score: 0.7473241689008997


Посмотрим на вес модели

In [ ]:
!du -shc output/quantization/*

679M	output/onnx_transforms/rubert-base-cased-sentiment.onnx
679M	total


Вывод:  по сравнению с моделью в формате ONNX скорость инференса модели стала  выше, метрики качества немного просели.Вес модели не уменьшился

# TensorRT

! Работает на тензорных ядрах, поэтому нужна тесла

Поэтому сначала поставим tensorrt, лучше из [архива](https://developer.nvidia.com/compute/machine-learning/tensorrt/secure/7.2.2/tars/TensorRT-7.2.2.3.Ubuntu-18.04.x86_64-gnu.cuda-11.1.cudnn8.0.tar.gz). Скачайте его и поместите на drive, затем выполните следующие команды

In [18]:
!nvidia-smi

Sun Apr 17 18:34:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [21]:
!uname -m

x86_64


In [25]:
!tar -xzvf /content/drive/MyDrive/TensorRT-7.2.2.3.Ubuntu-18.04.x86_64-gnu.cuda-11.1.cudnn8.0.tar.gz

TensorRT-7.2.2.3/
TensorRT-7.2.2.3/bin
TensorRT-7.2.2.3/lib
TensorRT-7.2.2.3/targets/
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/bin/
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/bin/trtexec
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvcaffe_parser.a
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvcaffe_parser.so.7.2.2
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvcaffe_parser.so.7
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvcaffe_parser.so
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvparsers_static.a
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvparsers.so.7.2.2
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libprotobuf-lite.a
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libprotobuf.a
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvparsers.so.7
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvparsers.so
TensorRT-7.2.2.3/targets/x86_64-linux-gnu/lib/libnvinfer_stat

In [26]:
!ls TensorRT-7.2.2.3
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:TensorRT-7.2.2.3/lib

bin   graphsurgeon  onnx_graphsurgeon  targets
data  include	    python	       TensorRT-Release-Notes.pdf
doc   lib	    samples	       uff


In [27]:
!python3 -m pip install TensorRT-7.2.2.3/python/tensorrt-7.2.2.3-cp37-none-linux_x86_64.whl

Processing ./TensorRT-7.2.2.3/python/tensorrt-7.2.2.3-cp37-none-linux_x86_64.whl


In [29]:
!python3 -m pip install TensorRT-7.2.2.3/uff/uff-0.6.9-py2.py3-none-any.whl

Processing ./TensorRT-7.2.2.3/uff/uff-0.6.9-py2.py3-none-any.whl


In [31]:
!python3 -m pip install TensorRT-7.2.2.3/graphsurgeon/graphsurgeon-0.4.5-py2.py3-none-any.whl

Processing ./TensorRT-7.2.2.3/graphsurgeon/graphsurgeon-0.4.5-py2.py3-none-any.whl


In [32]:
!python3 -m pip install TensorRT-7.2.2.3/onnx_graphsurgeon/onnx_graphsurgeon-0.2.6-py2.py3-none-any.whl

Processing ./TensorRT-7.2.2.3/onnx_graphsurgeon/onnx_graphsurgeon-0.2.6-py2.py3-none-any.whl
     |████████████████████████████████| 12.8 MB 12.9 MB/s 


In [36]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 13.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 6.0 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=1dd9b319b3fbdaccbfe94186da0fa4bab4bda60a23e904a1eb6b8d149cb423d7
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.3-py2.py3-none-any.whl size=64239 sha256=0c1fe0a8e4a1defe6ea055772f39a56e6371fe710665d664b2323867f36c4bc8
  Stored in directory: /root/.cache/pip/wheels/73/ea/cc/82049d3ddaad7909969ce140220641d70733db46ac09f4d8b4
Successfully built pycuda pytools


А теперь будем экспортировать модель в tensorrt

In [37]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

In [108]:
# logger to capture errors, warnings, and other information during the build and inference phases
TRT_LOGGER = trt.Logger()

def build_engine(onnx_file_path):
    # initialize TensorRT engine and parse ONNX model
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network()
    parser = trt.OnnxParser(network, TRT_LOGGER)
    
    # parse ONNX
    with open(onnx_file_path, 'rb') as model:
        print('Beginning ONNX file parsing')
        parser.parse(model.read())
    print('Completed parsing of ONNX file')
    # allow TensorRT to use up to 1GB of GPU memory for tactic selection
    builder.max_workspace_size = 1 << 30
    # we have only one image in batch
    builder.max_batch_size = 1
    # use FP16 mode if possible
    if builder.platform_has_fast_fp16:
        builder.fp16_mode = True

    # generate TensorRT engine optimized for the target platform
    print('Building an engine...')
    engine = builder.build_cuda_engine(network)
    context = engine.create_execution_context()
    print("Completed creating Engine")

    return engine, context

def build_engine_onnx(model_file):
  with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
      builder.max_workspace_size = 1 << 30
      # Load the Onnx model and parse it in order to populate the TensorRT network.
      with open(model_file, 'rb') as model:
          parser.parse(model.read())
      return builder.build_cuda_engine(network) 